가장 높은 ver1에 추가 feature들 적용
- onehot encoding 그대로 사용해보기

In [145]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [146]:
test_df = pd.read_csv('/content/drive/MyDrive/SW 중심대학 예선/Data/test.csv',index_col = 0)
train_df = pd.read_csv('/content/drive/MyDrive/SW 중심대학 예선/Data/train.csv',index_col = 0)

Data 전처리
  - outlier 처리
  - Null 처리

outlier 최빈값으로 처리


In [147]:
def outlier_func(data):
  for i in ['testelapse','surveyelapse','introelapse']:
    data.loc[data[i]>6000,i] = data[i].mode()[0]
  #age
  data.loc[data['age']>100,'age'] = data['age'].mode()[0]
  #familysize
  data.loc[data['familysize']>10,'familysize'] = data['familysize'].mode()[0]
  
  return data

introelapse drop

In [148]:
def drop_intro(data):
  data.drop(columns = 'introelapse',axis=1,inplace = True)
  return data

NULL 처리

In [149]:
def fill_na_mode(data):                 
    fill_null_list = list(data.columns)
    fill_null_list.remove('TIPI10')
    for col in fill_null_list:
        data[col].fillna(data[col].mode()[0],inplace=True)
    data['TIPI10'].fillna(1,inplace=True)

    return data

VCL6,VCL9,VCL12는 질문에 대한 validation check 질문으로 이 질문들에 1로 응답한 횟수가 높을수록 낮은 신뢰성을 보인다. 

In [150]:
def vcl_validation_featuring(data):
  data['VCL_validation'] =  data['VCL6']+data['VCL9']+ data['VCL12']
  return data

In [151]:
def tipi_engineering(data):
  data['Conscientiousness'] = (data['TIPI3']+(6-data['TIPI8']))/2
  data['Agreeableness'] = (data['TIPI7']+(6-data['TIPI2']))/2
  data['Neuroticism'] = (data['TIPI9']+(6-data['TIPI4']))/2
  data['openness'] = (data['TIPI5']+(6-data['TIPI10']))/2
  data['extraversion'] = (data['TIPI1']+(6-data['TIPI6']))/2
  return data

## Scaling & Encoding

country가 100명 이상인 국가를 제외한 data들은 country -> others로 처리

In [152]:
def country_func(data):
  temp = data['country'].value_counts().rename_axis('unique_values').reset_index(name='counts')
  country_list = list(temp[temp['counts']>100]['unique_values'].unique())
  data.loc[~data['country'].isin(country_list),'country'] = 'others'
  data = pd.get_dummies(data = data, columns = ['country'], prefix = 'country')
  return data

- one hot encoding 적용 여부

In [153]:
def onehot_encoding(data):
  onehot_list = ['education','urban', 'gender', 'engnat', 'hand', 'religion', 'orientation','voted', 'married', 'ASD']
  for col in onehot_list:
    data = pd.get_dummies(data = data, columns = [col],prefix = col)
  return data

이상치가 없는 데이터 columns

Standardscaler 사용

In [ ]:
# standardscaler_list = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11',
#        'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21',
#        'Q22', 'Q23', 'Q24', 'Q25', 'Q26','TIPI1', 'TIPI2', 'TIPI3', 'TIPI4', 'TIPI5', 'TIPI6',
#        'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10','familysize','VCL_validation','Conscientiousness', 
#        'Agreeableness', 'Neuroticism', 'openness','extraversion']

In [ ]:
# def standard_scaler(data):
#   # StandardScaler 선언 및 Fitting
#   sdscaler = StandardScaler()
#   sdscaler.fit(data[standardscaler_list])

#   # 데이터 변환
#   sdscaled_data = sdscaler.transform(data[standardscaler_list])

#   # 데이터 프레임으로 저장
#   sdscaled_data = pd.DataFrame(sdscaled_data,columns = standardscaler_list)
#   #data로 교체
#   data[standardscaler_list] = sdscaled_data[standardscaler_list]
#   return data

이상치 있는 columns

['introelapse', 'testelapse','surveyelapse']

Robustscaler 사용

In [154]:
def robust_scaler(data):
  robustscaler_list = ['testelapse','surveyelapse']
  # RobustScaler 선언 및 Fitting
  robust_scaler = RobustScaler()
  robust_scaler.fit(data[robustscaler_list])
  # 데이터 변환
  robustcaled_data = robust_scaler.transform(data[robustscaler_list])
  # 데이터 프레임으로 저장
  robustcaled_data = pd.DataFrame(robustcaled_data,columns = robustscaler_list)
  #data로 교체
  data[robustscaler_list] = robustcaled_data[robustscaler_list]
  return data

ageband

In [155]:
def ageband(data):
    data['ageBand'] = -1
    data.loc[(data['age'] > 12) & (data['age'] <= 18.0), 'ageBand'] = 1
    data.loc[(data['age'] > 18.0) & (data['age'] <= 24.0), 'ageBand'] = 2
    data.loc[(data['age'] > 24.0) & (data['age'] < 99999), 'ageBand'] = 3
    data.drop('age',axis=1,inplace=True)
    return data

In [156]:
def processing_data(data):      # 데이터 전처리 함수
  data = outlier_func(data)
  data= drop_intro(data)
  data= fill_na_mode(data)
  data= vcl_validation_featuring(data)
  data= country_func(data)
  data= onehot_encoding(data)
  data= robust_scaler(data)
  data= ageband(data)
  data = tipi_engineering(data)
  return data

In [157]:
train_df = processing_data(train_df)
test_df = processing_data(test_df)

In [158]:
train_df.to_csv('/content/drive/MyDrive/SW 중심대학 예선/Data/train_scaling_ver5_0821.csv')
test_df.to_csv('/content/drive/MyDrive/SW 중심대학 예선/Data/test_scaling_ver5__0821.csv')